In [5]:
import numpy as np

coreMatrix = np.array([[5.554, 0.252, 0.496, 0.237],
                       [0.580, 4.953, 0.467, 0.028],
                       [0.319, 0.372, 8.935, 0.520],
                       [0.043, 0.459, 0.319, 4.778]])

freeMembers = np.array([[0.442], [0.464], [0.979], [0.126]])

Задание I.

In [6]:
def gauss_method(coreMatrix, freeMembers):
    concMatrix = np.concatenate((coreMatrix, freeMembers), axis=1)

    for i in range(coreMatrix.shape[1]):
        j = i

        selRow = concMatrix[j]

        for i in range(j + 1, concMatrix.shape[0]):
            concMatrix[i] -= selRow * concMatrix[i, j] / concMatrix[j, j]

    x = np.zeros(concMatrix.shape[1] - 1)

    for i in reversed(range(concMatrix.shape[0])):
        for j in reversed(range(i, concMatrix.shape[1] - 1)):
            concMatrix[i] -= x[j] * concMatrix[i, j]

        concMatrix[i] /= concMatrix[i, i]
        x[i] = concMatrix[i, -1] / concMatrix[i, i]

    return x

print(gauss_method(coreMatrix, freeMembers))

[0.06800826 0.07675849 0.10342544 0.01156268]


In [7]:
def gauss_jordan_method(coreMatrix, freeMembers):
    concMatrix = np.concatenate((coreMatrix, freeMembers), axis=1)

    for i in range(coreMatrix.shape[1]):
        j = i

        selRow = concMatrix[j] / concMatrix[j, j]
        concMatrix[j] = selRow

        for i in range(j):
            concMatrix[i] -= selRow * concMatrix[i, j]

        for i in range(j + 1, concMatrix.shape[0]):
            concMatrix[i] -= selRow * concMatrix[i, j]

    return concMatrix[:, 4]

print(gauss_jordan_method(coreMatrix, freeMembers))

[0.06640598 0.07609387 0.10335724 0.01156268]


In [11]:
def jacobi_method(coreMatrix, freeMembers, eps):
    x = np.zeros(coreMatrix.shape[0])

    nextX = (freeMembers.T - (np.matmul(coreMatrix, x.T).T - coreMatrix.diagonal() * x)) / coreMatrix.diagonal()

    i = 1

    while i < 300 and np.max(np.abs((nextX - x))) > eps:
        x = nextX
        nextX = (freeMembers.T - (
                    np.matmul(coreMatrix, x.T).T - coreMatrix.diagonal() * x)) / coreMatrix.diagonal()

    return nextX

print(jacobi_method(coreMatrix, freeMembers, 1e-3))

[[0.06634421 0.07601978 0.10330396 0.01149752]]


In [12]:
def gauss_seidel_method(coreMatrix, freeMembers, eps):
    x = np.zeros(coreMatrix.shape[0])

    temp = np.copy(x)

    for i in range(coreMatrix.shape[0]):
        temp[i] = (freeMembers[i] - np.matmul(coreMatrix[i], temp .T) + coreMatrix[i, i] * temp [i]) / coreMatrix[i, i]

    nextX = temp

    i = 1

    while i < 300 and np.max(np.abs((nextX - x))) > eps:
        x = nextX

        temp = np.copy(x)
        for i in range(coreMatrix.shape[0]):
            temp[i] = (freeMembers[i] - np.matmul(coreMatrix[i], temp.T) + coreMatrix[i, i] * temp[i]) / coreMatrix[i, i]

        nextX = temp

    return nextX

print(gauss_seidel_method(coreMatrix, freeMembers, 1e-3))

[0.06639818 0.076089   0.10335811 0.01156316]


Задание II.

In [13]:
def conjugate_gradient_method(coreMatrix, freeMembers, eps):
    x = np.zeros(coreMatrix.shape[0])
    z = r = freeMembers.T - np.matmul(coreMatrix, x.T)

    i = 1

    while i < 300 and np.max(np.abs(r)) > eps:
        alpha = np.matmul(r, r.T) / np.matmul(np.matmul(coreMatrix, z.T).T, z.T)

        nextX = x + alpha * z
        nextR = r - alpha * np.matmul(coreMatrix, z.T).T

        beta = np.matmul(nextR, nextR.T) / np.matmul(r, r.T)

        nextZ = nextR + beta * z

        x = nextX
        r = nextR
        z = nextZ

    return x

print(conjugate_gradient_method(coreMatrix, freeMembers, 1e-9))

[[0.06640598 0.07609387 0.10335724 0.01156268]]


Задание III. 

In [25]:
def determinant(hilbert):
    det = 0

    if len(hilbert) == 2:
        return round(hilbert[0][0] * hilbert[1][1] - hilbert[0][1] * hilbert[1][0], 4)

    for c in range(len(hilbert)):
        det += round((((-1) ** c) * hilbert[0][c] * determinant(minor(hilbert, 0, c))), 4)

    return det


def minor(hilbert, i, j):
    minor = np.array([row[:j] + row[j + 1:]
                      for row in (hilbert[:i] + hilbert[i + 1:])])

    return minor


def inverse(hilbert):
    inv = [[(((-1) ** (r + c)) * determinant(minor(hilbert, r, c)))
            for c in range(len(hilbert))]
           for r in range(len(hilbert))]

    for i in range(len(inv)):
        for j in range(len(inv)):
            inv[i][j] = round(inv[i][j] / determinant(hilbert), 4)

    return inv


def hilbert(n):
    hilbert = [[round(1 / (i + j + 1), 4) for j in range(n)] for i in range(n)]

    print("H:")
    print(np.array(hilbert))
    
    hilbertInv = inverse(hilbert)
    hilbertInvInv = inverse(hilbertInv)

    print("H^-1:")
    print(np.array(hilbertInv))
    
    print("(H^-1)^-1:")
    print(np.array(hilbertInvInv))

    difference = np.array(hilbert) - np.array(hilbertInvInv)

    norm = 0

    for j in range(len(difference)):
        norm += difference[0][j]
        
    for i in range(1, len(difference)):
        
        temp = 0
        
        for j in range(len(difference)):
            temp += difference[i][j]

        norm = max(norm, temp)
    
    print("")
    print("L-norm: " + str(norm))

          
hilbert(3)

H:
[[1.     0.5    0.3333]
 [0.5    0.3333 0.25  ]
 [0.3333 0.25   0.2   ]]
H^-1:
[[  10.5   -41.75   34.75]
 [ -41.75  222.25 -208.5 ]
 [  34.75 -208.5   208.25]]
(H^-1)^-1:
[[0.897  0.4624 0.3132]
 [0.4624 0.3124 0.2356]
 [0.3132 0.2356 0.1884]]

L-norm: 0.1607


In [33]:
print("Gauss:                     " + str(gauss_method(coreMatrix, freeMembers)))
print("Gauss Jordan:              " + str(gauss_jordan_method(coreMatrix, freeMembers)))
print("Jacobi (1e-3):            " + str(jacobi_method(coreMatrix, freeMembers, 1e-3)))
print("Gauss Seidel (1e-3):       " + str(gauss_seidel_method(coreMatrix, freeMembers, 1e-3)))
print("Conjugate gradient (1e-3):" + str(conjugate_gradient_method(coreMatrix, freeMembers, 1e-3)))
print("")
print("Hilbert")
hilbert(3)

Gauss:                     [0.06800826 0.07675849 0.10342544 0.01156268]
Gauss Jordan:              [0.06640598 0.07609387 0.10335724 0.01156268]
Jacobi (1e-3):            [[0.06634421 0.07601978 0.10330396 0.01149752]]
Gauss Seidel (1e-3):       [0.06639818 0.076089   0.10335811 0.01156316]
Conjugate gradient (1e-3):[[0.06638226 0.07614913 0.10333829 0.01159617]]

Hilbert
H:
[[1.     0.5    0.3333]
 [0.5    0.3333 0.25  ]
 [0.3333 0.25   0.2   ]]
H^-1:
[[  10.5   -41.75   34.75]
 [ -41.75  222.25 -208.5 ]
 [  34.75 -208.5   208.25]]
(H^-1)^-1:
[[0.897  0.4624 0.3132]
 [0.4624 0.3124 0.2356]
 [0.3132 0.2356 0.1884]]

L-norm: 0.1607
